# Simple FRAP script

This script analyzes all Images in a Dataset, measuring the intensity in a named Channel within a ROI. The intensity is plotted over time.

### Import packages

In [ ]:
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numpy import array, int8
from skimage import feature
from scipy.ndimage import convolve
from scipy import misc
from scipy import ndimage
from omero.model import EllipseI
from omero.gateway import MapAnnotationWrapper

### Create a connection to the OMERO Server

In [ ]:
from omero.gateway import BlitzGateway
from getpass import getpass

HOST = 'outreach.openmicroscopy.org'
PORT = 4064
conn = BlitzGateway(raw_input("Username: "), getpass("OMERO Password: "), host=HOST, port=PORT)
conn.connect()

### Variables to edit

In [ ]:
dataset_id = 5220
channel_label = "GFP"

### Get Dataset and the Images it contains

In [ ]:
dataset = conn.getObject("Dataset", dataset_id)
images = []
for image in dataset.listChildren():
    print(image.id, image.name)
    images.append(image)
# Only use the first 2 images
images = images[:2]

### Define various helper methods we need

In [ ]:
def getEllipse(image):
    roi_service = conn.getRoiService()
    result = roi_service.findByImage(image.getId(), None)
    # Simply return any Ellipse we find...
    shape_id = None
    for roi in result.rois:
        print("ROI:", roi.getId().getValue())
        for s in roi.copyShapes():
            if type(s) == EllipseI:
                shape_id = s.id.val
    print("Shape:", shape_id)
    return shape_id

In [ ]:
def getChannelIndex(image, label):
    labels = image.getChannelLabels()
    if label in labels:
        idx = labels.index(label)
        print("Channel Index:", idx)
        return idx
    return 0

In [ ]:
def getMeanIntensities(image, the_c, shape_id):
    # Get pixel intensities for first Channel
    roi_service = conn.getRoiService()
    the_z = 0
    size_t = image.getSizeT()
    print('SizeT', size_t)
    meanvalues = []
    for t in range(size_t):
        stats = roi_service.getShapeStatsRestricted([shape_id], the_z, t, [the_c])
        meanvalues.append(stats[0].mean[the_c])
    print(meanvalues)
    return meanvalues

In [ ]:
def delete_old_annotations(image, namespace):
    to_delete = []
    for ann in image.listAnnotations(ns=namespace):
        to_delete.append(ann.id)
    if len(to_delete) > 0:
        print("Deleting old annotations", to_delete)
        conn.deleteObjects('Annotation', to_delete, wait=True)

In [ ]:
def add_map_annotation(image, meanvalues):
    # Add values as a Map Annotation on the image
    namespace = "demo.simple_frap_data"
    delete_old_annotations(image, namespace)
    key_value_data = [[str(t), str(value)] for t, value in enumerate(meanvalues)]
    map_ann = MapAnnotationWrapper(conn)
    map_ann.setNs(namespace)
    map_ann.setValue(key_value_data)
    map_ann.save()
    print("Added MapAnnotation", map_ann.id)
    image.linkAnnotation(map_ann)

### Process Images

Go through all the images, getting mean intensity values over time for an Ellipse on the named channel. Plot the values to see FRAP recovery over time.

In [ ]:
for image in images:
    print("Processing Image:", image.id)
    shape_id = getEllipse(image)
    if shape_id is None:
        continue
    ch_index = getChannelIndex(image, channel_label)
    meanvalues = getMeanIntensities(image, ch_index, shape_id)
    add_map_annotation(image, meanvalues)
    fig = plt.figure()
    plt.subplot(111)
    plt.plot(meanvalues)
    plt.show()

<br>
#### When you are completely finished running the notebook close the connection to OMERO server

In [ ]:
conn.close()